In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población media anual,513170
1,Huelva,Confirmados PDIA,62664
2,Huelva,Confirmados PDIA 14 días,7300
3,Huelva,Tasa PDIA 14 días,"1422,5305454332872"
4,Huelva,Confirmados PDIA 7 días,2785
5,Huelva,Tasa PDIA 7 dias,"542,7051464426993"
6,Huelva,Total Confirmados,62884
7,Huelva,Curados,50143
8,Huelva,Fallecidos,431


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  62664.0


/tmp/ipykernel_31773/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  18179.0


/tmp/ipykernel_31773/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]

/tmp/ipykernel_31773/242626003.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_31773/242626003.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_31773/242626003.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 151 personas en los últimos 7 días 

Un positivo PCR cada 59 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población media anual' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Rename columns
tabla.columns = ['Confirmados PDIA', 'Confirmados PDIA 14 días', 'Confirmados PDIA 7 días', 'Población']

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,62664.0,7300.0,2785.0,513170.0,542.705146,1422.530545,888.0
Huelva-Costa,36794.0,4144.0,1583.0,289548.0,546.714189,1431.196209,525.0
Huelva (capital),18179.0,2430.0,951.0,143837.0,661.165069,1689.412321,322.0
Sierra de Huelva-Andévalo Central,6944.0,1368.0,651.0,67391.0,966.004363,2029.944651,220.0
Condado-Campiña,18471.0,1772.0,545.0,156231.0,348.842419,1134.217921,141.0
Aljaraque,2454.0,339.0,143.0,21474.0,665.921580,1578.653255,58.0
Valverde del Camino,1534.0,299.0,114.0,12750.0,894.117647,2345.098039,57.0
Cartaya,2769.0,287.0,114.0,20083.0,567.644276,1429.069362,53.0
Lepe,3589.0,260.0,77.0,27880.0,276.183644,932.568149,29.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Hinojales,22.0,20.0,10.0,330.0,3030.303030,6060.606061,0.0
Rosal de la Frontera,167.0,68.0,37.0,1697.0,2180.318209,4007.071302,5.0
Cerro de Andévalo (El),301.0,69.0,47.0,2327.0,2019.767942,2965.191233,16.0
Jabugo,184.0,66.0,29.0,2260.0,1283.185841,2920.353982,8.0
Villanueva de las Cruces,42.0,11.0,7.0,387.0,1808.785530,2842.377261,1.0
Campofrío,92.0,18.0,3.0,713.0,420.757363,2524.544180,0.0
Aroche,247.0,72.0,30.0,3054.0,982.318271,2357.563851,3.0
Valverde del Camino,1534.0,299.0,114.0,12750.0,894.117647,2345.098039,57.0
Almonaster la Real,166.0,41.0,24.0,1788.0,1342.281879,2293.064877,5.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Almendro (El),77.0,8.0,1.0,840.0,119.047619,952.380952,0.0,0.125000
Lucena del Puerto,451.0,38.0,6.0,3261.0,183.992640,1165.286722,3.0,0.157895
San Bartolomé de la Torre,472.0,43.0,7.0,3761.0,186.120713,1143.312949,0.0,0.162791
Campofrío,92.0,18.0,3.0,713.0,420.757363,2524.544180,0.0,0.166667
Manzanilla,227.0,26.0,5.0,2118.0,236.071766,1227.573182,3.0,0.192308
Alosno,342.0,34.0,7.0,3933.0,177.981185,864.480041,1.0,0.205882
Paterna del Campo,380.0,60.0,14.0,3457.0,404.975412,1735.608909,1.0,0.233333
Palma del Condado (La),1713.0,110.0,26.0,10801.0,240.718452,1018.424220,4.0,0.236364
San Juan del Puerto,965.0,113.0,29.0,9411.0,308.150037,1200.722559,10.0,0.256637
